In [1]:
import pickle
import time
import random


def printv(*args, **kwargs):
    global verbose
    if verbose:
        print(*args, **kwargs)


def execute_command_assert_outputs(tc, cli, command, sleep):
    cli.execute_command(command)
    time.sleep(sleep)
    printv(command)
    tc.assert_eq(cli.out.outputs[0]["text"])


class TestCase:
    def __init__(self, fname, record):
        self.values = None
        self.fname = None
        if not record:
            with open('tests/'+fname, "rb") as f:
                self.values = iter(pickle.load(f))
            self.test_failed = 0
        else:
            self.fname = fname
        self.record_ = record
        self.to_record = []

    def assert_eq(self, thing):
        if not self.record_:
            val = next(self.values)
            if thing != val:
                print()
                print('<<<<<< WANTED')
                print(val)
                print('======')
                print(thing)
                print('>>>>>> GOT')
                self.test_failed += 1
            else:
                printv('Passed')
        else:
            self.to_record.append(thing)
            printv(thing)

    def assert_is(self, thing):
        if not self.record_:
            val = next(self.values)
            if thing is not val:
                print()
                print('<<<<<< WANTED')
                print(val)
                print('======')
                print(thing)
                print('>>>>>> GOT')
                self.test_failed += 1
            else:
                printv('Passed')
        else:
            self.to_record.append(thing)
            printv(thing)

    def record(self):
        if self.record_:
            with open('tests/'+self.fname, "wb") as f:
                pickle.dump(self.to_record, f)
        else:
            assert self.test_failed == 0, self.test_failed


def test_case(test_name, record=False, seed=None, verb=True, add_cli=False):
    global verbose
    verbose = verb

    def decorator(func):
        if not add_cli:

            def wrapper():
                tc = TestCase(test_name, record=record)
                if seed is not None:
                    random.seed(seed)
                func(tc)
                tc.record()
                print(f"Test {test_name} passed")

        else:

            def wrapper():
                try:
                    tc = TestCase(test_name, record=record)
                    if seed is not None:
                        random.seed(seed)
                    cli = CLI()
                    cli.execute_command("help close")
                    time.sleep(2)

                    func(tc, cli)

                    tc.record()
                    print(f"Test {test_name} passed")
                finally:
                    cli.execute_command("q")

        return wrapper

    return decorator


%run cli.ipynb
time.sleep(0.000001)
cli.execute_command("q")

In [2]:
genes = default_genes[BeeSpecies.FOREST]
forest_genes = Genes(**{k: (g, g) for k, g in genes.items()})
pf = Princess(forest_genes)
genes = default_genes[BeeSpecies.MEADOWS]
meadows_genes = Genes(**{k: (g, g) for k, g in genes.items()})
dm = Drone(meadows_genes)

In [3]:
def find_seed(pr_name, pr, dr_name, dr, pr_out_name, pr_spec, dr_out_name, dr_spec):
    print(pr, dr)
    queen = pr.mate(dr)
    ch = queen.die()
    size = len(ch)
    comma_fill = ", _" * (size - 2)
    seed = 0
    while True:
        random.seed(seed)
        ch = queen.die()
        if ch[0].genes.species == pr_spec and ch[1].genes.species == dr_spec:
            break
        seed += 1
    print(*ch[0].genes.species)
    print(*ch[1].genes.species)
    print(f"random.seed({seed})")
    print(f"queen = {pr_name}.mate({dr_name})")
    print(f"{pr_out_name}, {dr_out_name}{comma_fill} = queen.die()")
    print(f"{pr_out_name}.inspect()")
    print(f"{dr_out_name}.inspect()")
    print(f"printv({pr_out_name})")
    print(f"printv({dr_out_name})")
    print(f"tc.assert_eq({pr_out_name}.genes)")
    print(f"tc.assert_eq({dr_out_name}.genes)")

In [4]:
find_seed(
    "pf",
    pf,
    "dm",
    dm,
    "pcf",
    (BeeSpecies.COMMON, BeeSpecies.FOREST),
    "dcf",
    (BeeSpecies.COMMON, BeeSpecies.FOREST),
)

FOREST Princess MEADOWS Drone
COMMON FOREST
COMMON FOREST
random.seed(9)
queen = pf.mate(dm)
pcf, dcf, _, _ = queen.die()
pcf.inspect()
dcf.inspect()
printv(pcf)
printv(dcf)
tc.assert_eq(pcf.genes)
tc.assert_eq(dcf.genes)


In [5]:
verbose = True
tc = TestCase("mutation.test", record=False)
printv(pf, dm)

random.seed(9)
queen = pf.mate(dm)
pc, dc, _, _ = queen.die()
pc.inspect()
dc.inspect()
printv(pc)
printv(dc)
tc.assert_eq(pc.genes)
tc.assert_eq(dc.genes)

random.seed(86)
queen = pc.mate(dc)
pc, dc, _ = queen.die()
pc.inspect()
dc.inspect()
printv(pc)
printv(dc)
tc.assert_eq(pc.genes)
tc.assert_eq(dc.genes)

random.seed(1520)
queen = pc.mate(dm)
pcu, dcu, _, _ = queen.die()
pcu.inspect()
dcu.inspect()
printv(pcu)
printv(dcu)
tc.assert_eq(pcu.genes)
tc.assert_eq(dcu.genes)

random.seed(39)
queen = pcu.mate(dcu)
pcu, dcu, _ = queen.die()
pcu.inspect()
dcu.inspect()
printv(pcu)
printv(dcu)
tc.assert_eq(pcu.genes)
tc.assert_eq(dcu.genes)

random.seed(1131)
queen = pcu.mate(dc)
pncu, dncu, _ = queen.die()
pncu.inspect()
dncu.inspect()
printv(pncu)
printv(dncu)
tc.assert_eq(pncu.genes)
tc.assert_eq(dncu.genes)

random.seed(3)
queen = pncu.mate(dncu)
pn, dn, _ = queen.die()
pn.inspect()
dn.inspect()
printv(pn)
printv(dn)
tc.assert_eq(pn.genes)
tc.assert_eq(dn.genes)

tc.record()

FOREST Princess MEADOWS Drone
Princess
  species : COMMON, FOREST
  fertility : 2D, 3p
  lifespan : SHORTER, SHORTER
  speed : SLOWER, SLOWEST
Drone
  species : COMMON, FOREST
  fertility : 2D, 3p
  lifespan : SHORTER, SHORTER
  speed : SLOWEST, SLOWER
Passed
Passed
Princess
  species : COMMON, COMMON
  fertility : 3p, 3p
  lifespan : SHORTER, SHORTER
  speed : SLOWER, SLOWER
Drone
  species : COMMON, COMMON
  fertility : 2D, 2D
  lifespan : SHORTER, SHORTER
  speed : SLOWEST, SLOWEST
Passed
Passed
Princess
  species : CULTIVATED, COMMON
  fertility : 2D, 3p
  lifespan : SHORTER, shortest
  speed : SLOWER, fast
Drone
  species : CULTIVATED, COMMON
  fertility : 2D, 3p
  lifespan : SHORTER, shortest
  speed : SLOWER, fast
Passed
Passed
Princess
  species : CULTIVATED, CULTIVATED
  fertility : 2D, 3p
  lifespan : SHORTER, SHORTER
  speed : SLOWER, fast
Drone
  species : CULTIVATED, CULTIVATED
  fertility : 3p, 3p
  lifespan : SHORTER, shortest
  speed : SLOWER, SLOWER
Passed
Passed
Princ

In [6]:
from collections import Counter

l = []
for i in range(10000):
    _, dc, dc2, dc3 = pf.mate(dm).die()
    dc.inspect()
    dc2.inspect()
    dc3.inspect()
    l.append(dc.genes.species)
    l.append(dc2.genes.species)
    l.append(dc3.genes.species)
c = Counter(l)
s = sum(c.values())
for key in c:
    print(*key, c[key] / s)

FOREST MEADOWS 0.42883333333333334
MEADOWS FOREST 0.4222
FOREST COMMON 0.03643333333333333
COMMON FOREST 0.03746666666666667
MEADOWS COMMON 0.040466666666666665
COMMON MEADOWS 0.0346


In [7]:
from collections import Counter

l = []
princess = Princess(
    Genes(
        (BeeSpecies.COMMON, BeeSpecies.CULTIVATED),
        (BeeFertility(2), BeeFertility(2)),
        (BeeLifespan.NORMAL, BeeLifespan.NORMAL),
        (BeeSpeed.NORMAL, BeeSpeed.NORMAL),
    )
)
drone = Drone(
    Genes(
        (BeeSpecies.MEADOWS, BeeSpecies.MEADOWS),
        (BeeFertility(2), BeeFertility(2)),
        (BeeLifespan.NORMAL, BeeLifespan.NORMAL),
        (BeeSpeed.NORMAL, BeeSpeed.NORMAL),
    )
)
for i in range(50000):
    _, dc, dc2 = princess.mate(drone).die()
    dc.inspect()
    dc2.inspect()
    l.append(dc.genes.species)
    l.append(dc2.genes.species)
c = Counter(l)
s = sum(c.values())
for key in c:
    print(*key, c[key] / s)

COMMON MEADOWS 0.23517
MEADOWS CULTIVATED 0.24814
MEADOWS COMMON 0.23552
CULTIVATED MEADOWS 0.25138
COMMON CULTIVATED 0.00738
CULTIVATED COMMON 0.00763
CULTIVATED CULTIVATED 0.01478


In [11]:
verbose = True
tc = TestCase("inventory.test", record=False)
inv = Inventory(100)
inv.place_bees([pf, dm])
printv(inv)
inv.mate(0, 1)
printv(inv)
random.seed(0)
inv.place_bees(inv[0].take().die())
printv(inv)
tc.assert_eq(inv.empty_slots())
tc.assert_eq(inv.take(0).genes)
tc.assert_eq(inv.take(1).genes)

tc.record()

------ INV ------
0 FOREST Princess
1 MEADOWS Drone
------ INV ------
0 FOREST Queen, rem: 4
------ INV ------
0 FOREST Princess
1 MEADOWS Drone(3)
Passed
Passed
Passed


In [14]:
verbose = True
tc = TestCase("apiary.test", record=False)

random.seed(9)
r = Resources(honey=0)
api = Apiary('0', add_resources=r.add_resources)
api.put_princess(pf)
tc.assert_eq(api.princess.slot.genes)
printv(api)

api.put_drone(dm)
printv(api)
tc.assert_is(isinstance(api.princess.slot, Queen))
tc.assert_eq(api.princess.slot.genes)

api.update()
printv(api)
printv(r)
tc.assert_eq(r["honey"])


api.update()
api.update()
api.update()
printv(api)
tc.assert_eq(r["honey"])

api.update()
printv(api)
printv(r)
tc.assert_eq(api.inv.empty_slots())
tc.assert_eq(api.inv.take(0).genes)
tc.assert_eq(api.inv.take(1).genes)
tc.assert_eq(api.inv.take(2).genes)

tc.record()

Passed
------ APIARY 0 ------
Princess: FOREST Princess
Drone: Slot empty
  ------ INV ------
    Empty...
------ APIARY 0 ------
Princess: FOREST Queen, rem: 4
Drone: Slot empty
  ------ INV ------
    Empty...
Passed
Passed
------ APIARY 0 ------
Princess: FOREST Queen, rem: 3
Drone: Slot empty
  ------ INV ------
    Empty...
------ RESOURCES ------
honey: 1
wood: 1
Passed
------ APIARY 0 ------
Princess: FOREST Queen, rem: 0
Drone: Slot empty
  ------ INV ------
    Empty...
Passed
------ APIARY 0 ------
Princess: Slot empty
Drone: Slot empty
  ------ INV ------
  0 MEADOWS Princess
  1 FOREST Drone(2)
  2 MEADOWS Drone
------ RESOURCES ------
honey: 3
wood: 2
Passed
Passed
Passed
Passed


In [19]:
verbose = True
random.seed(0)
r = Resources(honey=0)
api = Apiary('0', add_resources=r.add_resources)

steps = 10000
updates = 4
for i in range(steps):
    api.put(pf)
    api.put(Drone(pf.genes))
    for i in range(updates + 2):
        api.update()
    api.inv.take(0)
    api.inv.take(1)
    api.inv.take(1)
    api.inv.take(1)
printv(r)
for res in ["honey", "wood"]:
    discrepancy = abs(r[res] / updates / steps - 0.5)
    printv(res, discrepancy)
    assert discrepancy < 0.1

------ RESOURCES ------
honey: 20029
wood: 19955
honey 0.0007249999999999757
wood 0.0011249999999999871


In [3]:
@test_case("cli.test", record=True, seed=0, verb=True, add_cli=True)
def cli_test(tc, cli):
    sleep_time = 1

    tc.assert_eq(len(cli.apiaries))
    tc.assert_eq(cli.inv.empty_slots())
    tc.assert_eq(cli.out.outputs[0]["text"])

    cli.execute_command("unshow")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])

    cli.execute_command("unshow")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])

    cli.execute_command("unshow")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])

    cli.execute_command("show resources")
    cli.execute_command("show inv")
    cli.execute_command("show apiary 0")
    cli.execute_command("forage")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])
    tc.assert_eq(cli.inv[0].slot.genes)
    tc.assert_eq(cli.inv[1].slot.genes)
    tc.assert_eq(cli.inv.empty_slots())

    cli.execute_command("put 0 0")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])
    tc.assert_is(cli.inv[0].is_empty())
    tc.assert_eq(cli.inv.empty_slots())
    tc.assert_eq(cli.apiaries[0].princess.slot.genes)

    cli.execute_command("put 0 1")
    time.sleep(sleep_time)
    # UGLY: this sleep should help a little with unstable updates
    time.sleep(0.5)
    tc.assert_eq(cli.out.outputs[0]["text"])
    tc.assert_is(cli.inv[1].is_empty())
    tc.assert_eq(cli.inv.empty_slots())
    tc.assert_is(isinstance(cli.apiaries[0].princess.slot, Queen))
    tc.assert_eq(cli.apiaries[0].princess.slot.genes)
    time.sleep(1)
    tc.assert_eq(cli.out.outputs[0]["text"])
    time.sleep(1)
    tc.assert_eq(cli.out.outputs[0]["text"])
    time.sleep(3)
    tc.assert_eq(cli.out.outputs[0]["text"])
    tc.assert_eq(cli.apiaries[0].inv[0].slot.genes)
    tc.assert_eq(cli.apiaries[0].inv[1].slot.genes)

    cli.execute_command("reput 0 0")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])
    tc.assert_is(cli.inv[0].is_empty())
    tc.assert_eq(cli.inv.empty_slots())
    tc.assert_eq(cli.apiaries[0].princess.slot.genes)

    cli.execute_command("reput 0 1")
    time.sleep(sleep_time)
    # UGLY: this sleep should help a little with unstable updates
    time.sleep(0.7)
    tc.assert_eq(cli.out.outputs[0]["text"])
    tc.assert_is(cli.inv[1].is_empty())
    tc.assert_eq(cli.inv.empty_slots())
    tc.assert_is(isinstance(cli.apiaries[0].princess.slot, Queen))
    tc.assert_eq(cli.apiaries[0].princess.slot.genes)
    time.sleep(2)
    tc.assert_eq(cli.out.outputs[0]["text"])
    time.sleep(1)
    tc.assert_eq(cli.out.outputs[0]["text"])
    time.sleep(3)
    tc.assert_eq(cli.out.outputs[0]["text"])
    tc.assert_eq(cli.apiaries[0].inv[0].slot.genes)
    tc.assert_eq(cli.apiaries[0].inv[1].slot.genes)

    cli.execute_command("reput 0 0 1")
    time.sleep(6)
    cli.execute_command("reput 0 0 1")
    time.sleep(6)
    tc.assert_eq(cli.resources["honey"])
    cli.execute_command("take 0 0 1")
    cli.execute_command("show inv 0")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])

    cli.execute_command("inspect 0")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])

    cli.execute_command("throw 0")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])

    cli.execute_command("swap 0 1")
    cli.execute_command("unshow")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])


cli_test()

1
100
------ RESOURCES ------
  Empty...

------ INV ------
  Empty...

------ APIARY 0 ------
Princess: Slot empty
Drone: Slot empty
  ------ INV ------
    Empty...

------ RESOURCES ------
  Empty...

------ INV ------
  Empty...

------ RESOURCES ------
  Empty...



------ RESOURCES ------
  Empty...

------ INV ------
0 MEADOWS Princess
1 MEADOWS Drone

------ APIARY 0 ------
Princess: Slot empty
Drone: Slot empty
  ------ INV ------
    Empty...

Genes(species=(<BeeSpecies.MEADOWS: 2>, <BeeSpecies.MEADOWS: 2>), fertility=(<BeeFertility.TWO: 2>, <BeeFertility.TWO: 2>), lifespan=(<BeeLifespan.SHORTER: 4>, <BeeLifespan.SHORTER: 4>), speed=(<BeeSpeed.SLOWEST: 0.3>, <BeeSpeed.SLOWEST: 0.3>))
Genes(species=(<BeeSpecies.MEADOWS: 2>, <BeeSpecies.MEADOWS: 2>), fertility=(<BeeFertility.TWO: 2>, <BeeFertility.TWO: 2>), lifespan=(<BeeLifespan.SHORTER: 4>, <BeeLifespan.SHORTER: 4>), speed=(<BeeSpeed.SLOWEST: 0.3>, <BeeSpeed.SLOWEST: 0.3>))
98
------ RESOURCES ------
  Empty...

------ INV --

In [5]:
@test_case("cli.build.test", record=False, seed=4, verb=True, add_cli=True)
def cli_build_test(tc, cli):
    sleep_time = 2

    tc.assert_eq(cli.out.outputs[0]["text"])
    cli.resources.add_resources({"wood": 4, "flowers": 4, "honey": 9})
    cli.render_event.set()
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])
    cli.execute_command("forage")
    cli.execute_command("forage")
    cli.execute_command("put 0 0 1")
    time.sleep(6)
    tc.assert_eq(cli.out.outputs[0]["text"])

    cli.execute_command("take 0 0..2")
    cli.execute_command("put 0 2 3")
    time.sleep(6)
    tc.assert_eq(cli.out.outputs[0]["text"])

    cli.execute_command("build apiary")
    cli.execute_command("show a 1")
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs[0]["text"])

    cli.execute_command("put 1 0 1")
    time.sleep(6)
    tc.assert_eq(cli.out.outputs[0]["text"])


cli_build_test()

Passed
Passed
Passed
Passed
Passed
Passed
Test cli.build.test passed


In [ ]:
@test_case("save.test", record=False, seed=0, verb=True, add_cli=True)
def save_test(tc, cli):
    execute_command_assert_outputs(tc, cli, "forage", 2)
    execute_command_assert_outputs(tc, cli, "save save_test", 1)
    execute_command_assert_outputs(tc, cli, "forage", 1)
    execute_command_assert_outputs(tc, cli, "load save_test", 1)
    execute_command_assert_outputs(tc, cli, "forage", 1)
    execute_command_assert_outputs(tc, cli, "load save_test", 1)


save_test()

In [ ]:
@test_case("help.test", record=False, seed=0, verb=True, add_cli=True)
def help_test(tc, cli):
    execute_command_assert_outputs(tc, cli, "help manual", 2)
    execute_command_assert_outputs(tc, cli, "help next", 2)
    execute_command_assert_outputs(tc, cli, "help prev", 2)
    execute_command_assert_outputs(tc, cli, "help go 5", 2)
    execute_command_assert_outputs(tc, cli, "help close", 2)
    execute_command_assert_outputs(tc, cli, "help manual", 2)
    execute_command_assert_outputs(tc, cli, "help help", 2)
    execute_command_assert_outputs(tc, cli, "help show", 2)
    execute_command_assert_outputs(tc, cli, "help close", 2)
    execute_command_assert_outputs(tc, cli, "help", 2)


help_test()

In [6]:
@test_case("put_multiple.test", record=False, seed=0, verb=True, add_cli=True)
def put_multiple_test(tc, cli):
    execute_command_assert_outputs(tc, cli, "forage", 2)
    execute_command_assert_outputs(tc, cli, "forage", 2)
    execute_command_assert_outputs(tc, cli, "put 0 0..3", 6)
    execute_command_assert_outputs(tc, cli, "put 0 0..1", 6)
    execute_command_assert_outputs(tc, cli, "reput 0 0 1 2", 6)
    execute_command_assert_outputs(tc, cli, "reput 0 0 1", 6)
    execute_command_assert_outputs(tc, cli, "put 0 3 2", 2)
    execute_command_assert_outputs(tc, cli, "put 0 2 3", 6)


put_multiple_test()

forage
Passed
forage
Passed
put 0 0..3
Passed
put 0 0..1
Passed
reput 0 0 1 2
Passed
reput 0 0 1
Passed
put 0 3 2
Passed
put 0 2 3

<<<<<< WANTED
------ RESOURCES ------
honey: 8
flowers: 6

------ INV ------
  Empty...

------ APIARY 0 ------
Princess: Slot empty
Drone: Slot empty
  ------ INV ------
  0 MEADOWS Princess
  1 MEADOWS Drone
  2 MEADOWS Drone
  3 MEADOWS Drone
  4 MEADOWS Princess
  5 MEADOWS Drone
  6 MEADOWS Drone

------ RESOURCES ------
honey: 8
flowers: 6

------ INV ------
  Empty...

------ APIARY 0 ------
Princess: Slot empty
Drone: Slot empty
  ------ INV ------
  0 MEADOWS Princess
  1 MEADOWS Drone
  2 MEADOWS Drone(2)
  3 MEADOWS Drone
  4 MEADOWS Princess
  5 MEADOWS Drone
  6 MEADOWS Drone

>>>>>> GOT


AssertionError: 1

In [ ]:
@test_case("take_overflow.test", record=False, seed=1, verb=True, add_cli=True)
def take_overflow_test(tc, cli):
    for i in range(50):
        cli.execute_command('forage')
    execute_command_assert_outputs(tc, cli, "put 0 0 1", 6)
    execute_command_assert_outputs(tc, cli, "take 0 0..3", 6)


take_overflow_test()

In [ ]:
@test_case("sort.test", record=False, seed=1, verb=True, add_cli=True)
def sort_test(tc, cli):
    execute_command_assert_outputs(tc, cli, "forage", 2)
    execute_command_assert_outputs(tc, cli, "forage", 2)
    execute_command_assert_outputs(tc, cli, "forage", 2)
    execute_command_assert_outputs(tc, cli, "forage", 2)
    execute_command_assert_outputs(tc, cli, "put 0 1 2", 5)
    execute_command_assert_outputs(tc, cli, "put 0 4 5", 5)
    execute_command_assert_outputs(tc, cli, "sort", 2)


sort_test()